In [ ]:
# Step 1: Install and import Dependencies
import os
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType

In [ ]:
# Step 2: Database Setup
def setup_database():
    """
    Creates and populates a SQLite database with employees and their managers.
    If the database file already exists, it will be deleted and recreated to ensure a clean state.
    """
    db_file = "company.db"
    # For demonstration purposes, we'll start with a fresh DB each time.
    if os.path.exists(db_file):
        os.remove(db_file)
        print("Removed existing database to apply new schema.")

    print(f"Creating a new database: {db_file}")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create the employees table. The 'id' column serves as the employeeID.
    cursor.execute('''
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        department TEXT NOT NULL,
        salary INTEGER NOT NULL
    )
    ''')
    print("Table 'employees' created.")

    # Create the manager mapping table.
    cursor.execute('''
    CREATE TABLE managerMapping (
        employeeID INTEGER NOT NULL,
        managerID INTEGER NOT NULL,
        FOREIGN KEY (employeeID) REFERENCES employees(id),
        FOREIGN KEY (managerID) REFERENCES employees(id)
    )
    ''')
    print("Table 'managerMapping' created.")

    # Insert sample employee data with a clear hierarchy.
    employees = [
        ('Charlie', 'Engineering', 120000), # id 1
        ('Frank', 'Engineering', 110000),   # id 2
        ('Alice', 'Engineering', 90000),    # id 3
        ('Grace', 'Human Resources', 100000),# id 4
        ('Bob', 'Human Resources', 65000),   # id 5
        ('David', 'Sales', 95000),          # id 6
        ('Eve', 'Sales', 80000)             # id 7
    ]
    cursor.executemany('INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)', employees)
    print(f"{len(employees)} records inserted into 'employees'.")

    # Insert manager-reportee relationships.
    # (employeeID, managerID)
    manager_mappings = [
        (3, 1), # Alice reports to Charlie
        (2, 1), # Frank reports to Charlie
        (5, 4), # Bob reports to Grace
        (7, 6)  # Eve reports to David
    ]
    cursor.executemany('INSERT INTO managerMapping (employeeID, managerID) VALUES (?, ?)', manager_mappings)
    print(f"{len(manager_mappings)} records inserted into 'managerMapping'.")

    conn.commit()
    conn.close()
    print("Database setup complete.")

In [ ]:
setup_database()

In [ ]:
# Step 3: Initialize SQLDatabase, LLM, Create SQL Agent
db = SQLDatabase.from_uri("sqlite:///company.db")
llm = ChatOpenAI(model="gpt-3.5-turbo")
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, # Set to True to see the agent's thought process
    handle_parsing_errors=True # Handles errors if the LLM output cannot be parsed
)

In [ ]:
# Step 4: Define the query_database function
def query_database(natural_language_query):
    """
    Takes a natural language query, uses the LangChain agent to get an answer,
    and prevents modification queries like UPDATE or DELETE.
    """
    print(f"\nProcessing query: '{natural_language_query}'")

    # Check for forbidden keywords to prevent modifications
    forbidden_keywords = ['update', 'delete', 'insert', 'drop', 'alter', 'create', 'modify']
    query_lower = natural_language_query.lower()
    if any(keyword in query_lower for keyword in forbidden_keywords):
        print("\nI can only execute SELECT queries. Can't execute updates and deletes.")
        return

    try:
        # The agent is invoked with the natural language query
        result = agent_executor.invoke(natural_language_query)
        final_answer = result.get('output', "Could not find the answer.")
        
        # Extract and print the generated SQL query for clarity
        if 'intermediate_steps' in result and result['intermediate_steps']:
             for step in result['intermediate_steps']:
                if isinstance(step[0].tool_input, str) and "SELECT" in step[0].tool_input.upper():
                    print(f"\nGenerated SQL Query:\n---\n{step[0].tool_input}\n---")
                    break

        print(f"\nAnswer: {final_answer}")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("This might be my best guess, or I am unable to predict exactly.")

In [ ]:
# Step 5: Testing the SQL Agent with some queries!
# 1. A straightforward query
query_database("How many employees are in the Engineering department?")

In [ ]:
# 2. A query that requires aggregation
query_database("What is the total salary for the Sales department?")

In [ ]:
# 3. A query that lists data
query_database("List all employees with a salary greater than 90000")

In [ ]:
# 4. A query that requires ordering
query_database("Who is the highest paid employee?")

In [ ]:
# 5. A query that is outside the scope of the database schema
query_database("Who is the smartest employee?")

In [ ]:
# 6. A vague query where the agent might have to guess
query_database("Tell me about the employees.")

In [ ]:
# 7. Simple manager lookup
query_database("Who is the manager of Alice?")

In [ ]:
# 8. Listing direct reportees
query_database("List all employees who report to Charlie")

In [ ]:
# 9. Finding a manager's manager (multi-level)
query_database("Who is the manager of manager of Alice - 2nd level manager?") # This should return no one as Charlie is top-level in this data

In [ ]:
# 10. A query that tries to modify data (should be blocked)
query_database("Delete the employee named Bob")

In [ ]:
if __name__ == "__main__":
    # Allow for user input
    print("\n--- Interactive Mode ---")
    print("Enter your question in natural language (or type 'exit' to quit).")
    while True:
        user_input = input("> ")
        if user_input.lower() == 'exit':
            break
        query_database(user_input)